## 连接图数据库

In [1]:
from py2neo import Graph

# 连接图数据库neo4j
graph = Graph("bolt://68.79.23.135:5432", auth=("neo4j", "123456"))
graph

Graph('bolt://neo4j@68.79.23.135:5432')

In [2]:
clabel = {'产品名称': '名称',
 '产品代码': '名称',
 '产品登记编码': '名称',
 '理财币种': '属性',
 '产品类型': '属性',
 '募集方式': '属性',
 '运作方式': '属性',
 '税款': '属性',
 '理财计划份额': '发行',
 '理财计划份额面值': '发行',
 '发行规模': '发行',
 '本金及理财收益': '收益',
 '业绩比较基准': '收益',
 '估值日': '收益',
 '巨额赎回': '认购申购与赎回',
 '认购/申购上限': '认购申购与赎回',
 '认购、申购及赎回方式': '认购申购与赎回',
 '认购起点': '认购申购与赎回',
 '认购期': '认购申购与赎回',
 '认购登记日': '认购申购与赎回',
 '理财计划申购和赎回': '认购申购与赎回',
 '费用': '认购申购与赎回',
 '理财计划到期日': '开始和结束时间',
 '理财计划成立日': '开始和结束时间',
 '投资策略': '投资'}

name_label = {'产品名称':'name', '产品代码':'code', '产品登记编码':'outer_code', '发行规模': {'上限':'up', '下限':'down'},
             '业绩比较基准':{'上限':'up', '下限':'down'}, '认购/申购上限':{'上限':'up', '单位':'unit'}, 
             '认购起点':{'下限':'down', '单位':'unit'}, '认购期':{'上限':'up', '下限':'down'},
              '理财计划申购和赎回': {'周期':'period', '单位':'unit'}, }

In [4]:
graph.query("Match (n:`名称`)-[rel:`属性`]-(m:`属性`) Where m.name='税款' Return distinct n.name, m.value").data()

[{'n.name': '招商银行美元金葵季季开6号理财计划',
  'm.value': '本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求外，投资者应缴纳的税收由投资者负责，产品管理人不承担代扣代缴或纳税的义务。理财计划在资产管理、运营、处置过程中产生的收入，根据国家相关法律法规应缴纳增值税（含增值税附加税费，下同）及/或其他税费的，即使本产品管理人被税务机关认定为纳税义务人，该等增值税及/或其他税费仍属于应由理财计划承担的理财计划税费，由本产品管理人申报和缴纳，该等税款直接从理财计划中扣付缴纳。详细内容见“理财计划税费”。'},
 {'n.name': '招商银行美元金葵季季开5号理财计划',
  'm.value': '本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求外，投资者应缴纳的税收由投资者负责，产品管理人不承担代扣代缴或纳税的义务。理财计划在资产管理、运营、处置过程中产生的收入，根据国家相关法律法规应缴纳增值税（含增值税附加税费，下同）及/或其他税费的，即使本产品管理人被税务机关认定为纳税义务人，该等增值税及/或其他税费仍属于应由理财计划承担的理财计划税费，由本产品管理人申报和缴纳，该等税款直接从理财计划中扣付缴纳。详细内容见“理财计划税费”。'},
 {'n.name': '招商银行美元金葵季季开4号理财计划',
  'm.value': '本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求外，投资者应缴纳的税收由投资者负责，产品管理人不承担代扣代缴或纳税的义务。理财计划在资产管理、运营、处置过程中产生的收入，根据国家相关法律法规应缴纳增值税（含增值税附加税费，下同）及/或其他税费的，即使本产品管理人被税务机关认定为纳税义务人，该等增值税及/或其他税费仍属于应由理财计划承担的理财计划税费，由本产品管理人申报和缴纳，该等税款直接从理财计划中扣付缴纳。详细内容见“理财计划税费”。'},
 {'n.name': '招商银行美元金葵季季开3号理财计划',
  'm.value': '本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求外，投资者应缴纳的税收由投资者负责，产品管理人不承担代扣

In [165]:
def greater_or_smaller(string):
    if string == '大于':
        return '>'
    elif string == '大于等于':
        return '>='
    elif string == '小于':
        return '<'
    elif string == '小于等于':
        return '<='
    elif string == '等于':
        return '='
    elif sring == '不等于':
        return '<>'
    
    
def match_is_exist(obj_dict, attri_dict, l=clabel, nl=name_label, graph=graph):
    # 首先遍历合并相同的属性
    after_obj_dict = {}
    for obj in obj_dict:
        key = list(obj.keys())[0]
        value = list(obj.values())[0]
        if key in after_obj_dict:
            after_obj_dict[key].append(value)
        else:
            after_obj_dict[key] = [value]
    print(after_obj_dict)
    
    # 同个属性间取“or”，不同属性间取“and”
    for key in list(after_obj_dict.keys()):
        label = l[key]
        value = after_obj_dict[key]
        
        # 如果类别不等于名称
        if label != '名称':
            match_ = "MATCH (m:`{0}`)-[rel:`{0}` {{links:'{1}为'}}]-(n)".format(label, key)
            return_ = "RETURN n.name"
            where_list = []
            
            # 如果不在特殊的标签
            if key not in nl:
                for v in value:
                    where_list.append("m.value = '{}'".format(v))
                where_ = "WHERE" + " " + " or ".join(where_list)
                
            elif key in ['发行规模', '业绩比较基准', '认购期']:
                # 记录需要判断非空的属性
                not_null = []
                for v in value:
                    filter_ =  "toFloat({})".format("m.{}".format(nl[key][v[0]])) + " " + greater_or_smaller(v[1]) +\
                             " " + str(v[2])
                    where_list.append(filter_)
                    not_null.append("m.{}".format(nl[key][v[0]]) + " is not null")
                # 合并
                not_null = " and ".join(not_null)
                # 这里同一属性间取“and”
                where_ = "WHERE" + " (" + " and ".join(where_list) + ") and (" + not_null + ")"
                
            elif key in ['认购/申购上限', '认购起点', '理财计划申购和赎回']:
                # 记录需要判断非空的属性
                not_null = []
                for v in value:
                    filter_ = "toFloat({})".format("m.{}".format(nl[key][list(nl[key].keys())[0]])) + " " +\
                            greater_or_smaller(v[0]) + " " + str(v[1]) + " and " + "m.unit = '{}'".format(v[2])
                    where_list.append(filter_)
                    not_null.append("m.{}".format(nl[key][list(nl[key].keys())[0]]) + " is not null")
                # 合并
                not_null = " and ".join(not_null)
                # 这里同一属性间取“and”
                where_ = "WHERE" + " (" + " and ".join(where_list) + ") and (" + not_null + ")"
    
        else:
            match_ = "MATCH (n:`名称`)"
            return_ = "RETURN n.name"
            where_list = []
            
            for v in value:
                where_list.append("n.{} = '.*{}.*'".format(nl[key], v))
            where_ = "WHERE" + " " + " or ".join(where_list)
            
        print(match_+" "+where_+" "+return_)


In [166]:
 match_is_exist([{'产品名称': '青葵'}, 
                 {'产品名称': '招商银行'},
                 {'产品代码': 'C18654'},
                 {'认购/申购上限': ['小于等于','40000000', '元']},
                 {'认购/申购上限': ['大于等于','10000000', '元']}], None)

{'产品名称': ['青葵', '招商银行'], '产品代码': ['C18654'], '认购/申购上限': [['小于等于', '40000000', '元'], ['大于等于', '10000000', '元']]}
MATCH (n:`名称`) WHERE n.name = '.*青葵.*' or n.name = '.*招商银行.*' RETURN n.name
MATCH (n:`名称`) WHERE n.code = '.*C18654.*' RETURN n.name
MATCH (m:`认购申购与赎回`)-[rel:`认购申购与赎回` {links:'认购/申购上限为'}]-(n) WHERE (toFloat(m.up) <= 40000000 and m.unit = '元' and toFloat(m.up) >= 10000000 and m.unit = '元') and (m.up is not null and m.up is not null) RETURN n.name


[{'理财币种': '人民币'}, {'理财币种': '美元'}]

In [64]:
match_graph([{'理财币种': '人民币'}, {'产品名称':'青葵'}], ['募集方式', '产品代码', '产品名称'])

MATCH (m:`投资`)-[rel:`投资` {links:'理财币种为'}]-(n) WHERE m.name = '理财币种' and m.value = '人民币' RETURN n.name
MATCH (n:`名称`) WHERE n.name =~ '.*青葵.*' RETURN n.name


{'理财币种:人民币': ['招商银行青葵系列两年定开012号理财计划',
  '招商银行青葵系列两年定开011号理财计划',
  '招商银行青葵系列两年定开010号理财计划',
  '招商银行青葵系列两年定开009号理财计划',
  '招商银行青葵系列两年定开008号理财计划',
  '招商银行青葵系列两年定开007号理财计划',
  '招商银行青葵系列两年定开006号理财计划',
  '招商银行青葵系列两年定开005号理财计划',
  '招商银行青葵系列两年定开004号理财计划',
  '招商银行青葵系列两年定开003号理财计划',
  '招商银行青葵系列两年定开002号理财计划',
  '招商银行青葵系列两年定开001号理财计划',
  '招商银行青葵系列一年定开004号理财计划',
  '招商银行青葵系列一年半定开012号理财计划',
  '招商银行青葵系列一年半定开011号理财计划',
  '招商银行青葵系列一年半定开010号理财计划',
  '招商银行青葵系列一年半定开009号理财计划',
  '招商银行青葵系列一年半定开008号理财计划',
  '招商银行青葵系列一年半定开007号理财计划',
  '招商银行青葵系列一年半定开006号理财计划',
  '招商银行青葵系列一年半定开005号理财计划',
  '招商银行零售青葵系列半年定开4号理财计划',
  '招商银行零售青葵系列半年定开3号理财计划',
  '招商银行零售青葵系列半年定开2号理财计划',
  '招商银行零售青葵系列一年定开9号理财计划',
  '招商银行零售青葵系列一年定开8号理财计划',
  '招商银行零售青葵系列一年定开1号理财计划',
  '招商银行零售青葵系列一年半定开5号理财计划',
  '招商银行零售青葵系列一年半定开4号理财计划',
  '招商银行零售青葵系列一年半定开1号理财计划',
  '招商银行年年享6号理财计划',
  '招商银行年年享5号理财计划'],
 '产品名称:青葵': ['招商银行零售青葵系列一年半定开1号理财计划',
  '招商银行零售青葵系列一年半定开4号理财计划',
  '招商银行零售青葵系列一年半定开5号理财计划',
  '招商银行零售青葵系列一年定开1号理财计划',
  '招商银行零售青葵系列一年定开8号理财计划',
  '招商银行零售青葵系列一

### 1、查询是否存在(则只针对名称属性进行查询，发现不是名称属性的一律跳过)

In [32]:
match_graph([{'产品名称': '招商银行青葵系列两年定开005号理财计划'}, {'产品名称': '青葵系列'},\
             {'产品名称': '小儿感冒灵计划'}, {'产品登记编码': 'C1030820000104'}, {'投资策略':'一年期'}], ['是否存在'])

({'招商银行青葵系列两年定开005号理财计划': [{'n': Node('名称', code='301060', id='36', name='招商银行青葵系列两年定开005号理财计划', outer_code='C1030820000104')}],
  '青葵系列': [{'n': Node('名称', code='301037', id='14', name='招商银行零售青葵系列一年半定开1号理财计划', outer_code='C1030819000278')},
   {'n': Node('名称', code='301040', id='15', name='招商银行零售青葵系列一年半定开4号理财计划', outer_code='C1030819000799')},
   {'n': Node('名称', code='301041', id='16', name='招商银行零售青葵系列一年半定开5号理财计划', outer_code='C1030819000993')},
   {'n': Node('名称', code='301025', id='17', name='招商银行零售青葵系列一年定开1号理财计划', outer_code='C1030819000030')},
   {'n': Node('名称', code='301032', id='18', name='招商银行零售青葵系列一年定开8号理财计划', outer_code='C1030819000923')},
   {'n': Node('名称', code='301033', id='19', name='招商银行零售青葵系列一年定开9号理财计划', outer_code='C1030819000949')},
   {'n': Node('名称', code='301002', id='20', name='招商银行零售青葵系列半年定开2号理财计划', outer_code='C1030819000064')},
   {'n': Node('名称', code='301003', id='21', name='招商银行零售青葵系列半年定开3号理财计划', outer_code='C1030819000111')},
   {'n': Node('名称', code='30

In [33]:
graph.query("MATCH (n:`投资`)-[rel1:`投资` {links:'理财币种为'}]-(m)-[rel2:`收益` {links:'业绩比较基准为'}]-(p) \
            WHERE n.name = '理财币种' and n.value = '美元' and m.name =~ '.*' RETURN m,rel2,p").data()

[{'m': Node('名称', code='2112', id='13', name='招商银行美元金葵季季开6号理财计划', outer_code='C1030820000437'),
  'rel2': 收益(Node('名称', code='2112', id='13', name='招商银行美元金葵季季开6号理财计划', outer_code='C1030820000437'), Node('收益', id='116', name='业绩比较基准', value='首个投资周期年化1.5%-2.1%，业绩比较基准不代表本产品的未来表现和实际收益，或招商银行对本产品进行的收益承诺。招商银行有权根据市场情况进行调整，该调整将通过一网通上“信息公告”进行信息披露。'), links='业绩比较基准为'),
  'p': Node('收益', id='116', name='业绩比较基准', value='首个投资周期年化1.5%-2.1%，业绩比较基准不代表本产品的未来表现和实际收益，或招商银行对本产品进行的收益承诺。招商银行有权根据市场情况进行调整，该调整将通过一网通上“信息公告”进行信息披露。')},
 {'m': Node('名称', code='2110', id='12', name='招商银行美元金葵季季开5号理财计划', outer_code='C1030820000314'),
  'rel2': 收益(Node('名称', code='2110', id='12', name='招商银行美元金葵季季开5号理财计划', outer_code='C1030820000314'), Node('收益', id='115', name='业绩比较基准', value='首个投资周期年化1.6%-2.2%，业绩比较基准不代表本产品的未来表现和实际收益，或招商银行对本产品进行的收益承诺。招商银行有权根据市场情况进行调整，该调整将通过一网通上“信息公告”进行信息披露。'), links='业绩比较基准为'),
  'p': Node('收益', id='115', name='业绩比较基准', value='首个投资周期年化1.6%-2.2%，业绩比较基准不代表本产品的未来表现和实际收益，或招商银行对本产品进行的收益承诺。招商银行有权根据市场情况进行调整，